In [ ]:
"""Below is the XGBoost regression model used for predictive analytics, followed by some of the previous versions"""

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
import time

start_time = time.time()

# Split the data into features (X) and target (y)
X = df.drop('Order_Demand', axis=1)
y = df['Order_Demand'].astype(float)

# Assign sample weights based on quantiles of demand
quantiles = np.percentile(y, [25, 50, 75])
weights = np.ones_like(y)
# Assign higher weights to lower and upper quantiles (underrepresented demand levels)
weights[y <= quantiles[0]] = 1.5  # 1st quartile (low demand)
weights[y >= quantiles[2]] = 1.5  # 4th quartile (high demand)

# Split the data into training and test sets
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, weights, test_size=0.2, random_state=42)

# Define hyperparameter space for tuning
param_grid = {
    'n_estimators': [100, 150, 200],  # Number of trees
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
    'max_depth': [3, 4, 5],  # Max depth of trees
    'min_child_weight': [1, 2, 3],  # Minimum sum of instance weight needed in a child
    'subsample': [0.8, 0.9, 1.0],  # Subsample ratio of the training data
    'colsample_bytree': [0.8, 0.9, 1.0],  # Subsample ratio of features per tree
    'reg_alpha': [0.01, 0.05, 0.1],  # L1 regularization term
    'reg_lambda': [0.5, 1.0, 1.5]    # L2 regularization term
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=param_grid, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_squared_error', 
    verbose=1, 
    n_jobs=1
)

# Fit the model with the training data and weights
random_search.fit(X_train, y_train, sample_weight=weights_train)

# Extract the best model
best_model = random_search.best_estimator_

# Evaluate the model
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)

# Predict using the best model
y_pred = best_model.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
relative_rmse = (rmse / y_test.mean()) * 100
mean_demand = y_test.mean()

# Output results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Relative RMSE: {relative_rmse:.2f}%")
print(f"Mean Demand: {mean_demand}")
print(f"Train Score: {train_score}")
print(f"Test Score: {test_score}")

# Additional evaluation
evaluation_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
evaluation_df.head()

print(f"Execution time: {time.time() - start_time} seconds")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Mean Squared Error (MSE): 1.601085067077502
Root Mean Squared Error (RMSE): 1.2653399017961546
Relative RMSE: 26.55%
Mean Demand: 4.76569143824976
Train Score: 0.7903385660989202
Test Score: 0.7883476063211605
Execution time: 317.209349155426 seconds


In [ ]:
"""combined weight loss with randomsearchCV k fold validation, 0.1:26.5%. best result"""

In [ ]:
"""
Relative RMSE of 27% is generally considered acceptable, for this specific use case it could require some checks:

Financial Impact Assessment with Cost of Stockouts
Inventory Management KPIs for Service Level and Inventory Turnover
Safety Stock Calculation
Impact on Lead Time
Sensitivity Analysis

In retail, a relative RMSE between 10% and 30% is generally considered acceptable, 
though depends on the complexity of the product mix and seasonality.
For an outdoor equipment shop, where demand can be highly seasonal (e.g., spikes during spring and summer, as well as before holidays), a relative RMSE near 25% is reasonable.

Product Types: Outdoor equipment can range from low-cost, high-volume items (like camping gear) to high-cost, low-volume items (like tents, kayaks, and bikes). Forecasting for high-cost items usually tolerates slightly higher errors due to the lower volume of purchases, while low-cost, high-volume items typically require more precise forecasting.
Seasonality: Because the demand for outdoor equipment is highly seasonal, seasonality adjustments are crucial. A relative RMSE of ~26.8% might still be acceptable during off-season times if it's more accurate during peak seasons.

Run Simulations to estimate the financial impact of forecasting errors on inventory costs and lost sales due to stockouts or overstocking. (NOT WITHIN SCOPE OF PROJECT)
Benchmark RMSE against any available industry data or past forecasting methods (moving averages, ARIMA, etc.) to see how much improvement the model brings.
Adjust Safety Stock based on the RMSE to balance procurement efficiency with service level.
"""

In [ ]:
"""
below are some previous versions of simpler XGBoost models, they have marginally worse accuracy but much faster run times. 
It is ucertain how any of these perform on larger datsets, testing of this is not within the scope of this project, so I have left them as part of the notebook. 
"""

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import time

start_time = time.time()

# Split the data into features (X) and target (y)
X = df.drop('Order_Demand', axis=1)
y = df['Order_Demand'].astype(float)

# Assign sample weights based on quantiles of demand
quantiles = np.percentile(y, [25, 50, 75])
weights = np.ones_like(y)
# Assign higher weights to lower and upper quantiles (underrepresented demand levels)
weights[y <= quantiles[0]] = 1.5  # 1st quartile (low demand)
weights[y >= quantiles[2]] = 1.5  # 4th quartile (high demand)

# Split the data into training and test sets
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, weights, test_size=0.2, random_state=42)

# Train an XGBoost model with sample weights
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train, sample_weight=weights_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
relative_rmse = (rmse / y_test.mean()) * 100
mean_demand = y_test.mean()

# Output results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Relative RMSE: {relative_rmse:.2f}%")
print(f"Mean Demand: {mean_demand}")
print(f"Train Score: {train_score}")
print(f"Test Score: {test_score}")

# Additional evaluation
evaluation_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
evaluation_df.head()
print(f"Execution time: {time.time() - start_time} seconds")

Mean Squared Error (MSE): 1.632817576975212
Root Mean Squared Error (RMSE): 1.277817505348558
Relative RMSE: 26.81%
Mean Demand: 4.76569143824976
Train Score: 0.7864026698230477
Test Score: 0.7841527875601891
Execution time: 2.983020544052124 seconds


In [ ]:
"""
weight loss function, with log transformation to highly skewed order demand
took relative RMSE down to 0.1: 26%
"""

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import time

start_time = time.time()

# Split the data into features (X) and target (y)
X = df.drop('Order_Demand', axis=1)
y = df['Order_Demand'].astype(float)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 150, 200],  # Narrower range
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],  # Fine-tuning smaller ranges
    'min_child_weight': [1, 2, 3],  # Narrower range
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0.01, 0.05, 0.1],  # More regularization
    'reg_lambda': [0.5, 1.0, 1.5]    # More regularization
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

random_search = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=param_grid, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_squared_error', 
    verbose=1, 
    n_jobs=1
)
random_search.fit(X_train, y_train)

# Train the best model from GridSearchCV with early stopping
best_model = random_search.best_estimator_

# fit with early stopping using the validation set
best_model.fit(X_train, y_train)

# Evaluate the model
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)

y_pred = best_model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
relative_rmse = (rmse / y_test.mean()) * 100
mean_demand = y_test.mean()

# Output the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Relative RMSE: {relative_rmse:.2f}%")
print(f"Mean Demand: {mean_demand}")

# Additional evaluation
evaluation_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
evaluation_df.head()
print(f"Execution time: {time.time() - start_time} seconds")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Mean Squared Error (MSE): 1.6121421196100456
Root Mean Squared Error (RMSE): 1.2697015868344994
Relative RMSE: 26.64%
Mean Demand: 4.76569143824976
Execution time: 257.434823513031 seconds


In [ ]:
"""
just randomsearchCV, 0.1:26.6%
"""